In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：107


['https://www.1aauto.com/1a-auto-parts/b/101',
 'https://www.1aauto.com/accu-form-parts/b/260',
 'https://www.1aauto.com/acdelco-parts/b/3',
 'https://www.1aauto.com/acdelco-and-gm-parts/b/62',
 'https://www.1aauto.com/armor-all-parts/b/202',
 'https://www.1aauto.com/arnott-parts/b/15',
 'https://www.1aauto.com/atlantic-automotive-engineering-parts/b/322',
 'https://www.1aauto.com/audi-parts/b/49',
 'https://www.1aauto.com/auto-7-parts/b/24',
 'https://www.1aauto.com/auto-custom-carpets-parts/b/95',
 'https://www.1aauto.com/autolite-parts/b/103',
 'https://www.1aauto.com/bbt-automotive-components-parts/b/308',
 'https://www.1aauto.com/beck-arnley-parts/b/16',
 'https://www.1aauto.com/borgeson-parts/b/29',
 'https://www.1aauto.com/bosch-parts/b/220',
 'https://www.1aauto.com/cardone-reman-parts/b/253',
 'https://www.1aauto.com/cardone-select-parts/b/117',
 'https://www.1aauto.com/carrand-parts/b/194',
 'https://www.1aauto.com/catco-parts/b/263',
 'https://www.1aauto.com/cateran-inc-part

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('page', '')
dict_param['sort'] = 'name+asc'

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'
    
list_url, i = [], 1
while True:
    b = 0
    while True:
        b += 1
        print(b, i)
        try:
            resp = requests.get(f'{url}page={i}',
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))
        
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                list_href = html.xpath('//ul[@id="serp-list"]/li/div/div[@class="desktop-product-results"]/descendant::h4/a/@href')
                if list_href:
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    list_url += [f'''https://www.1aauto.com{href.strip()}''' for href in list_href]

    # = = = = = = = = = = = = = = =
    
    i += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//a[@class="pagination-button pagination-button-next"]'):
        break

resp

1 1


<Response [200]>

In [6]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                       'Url': list_url})

output = output.sort_values(by=['No'],
                            ascending=[True],
                            ignore_index=True)

output.to_excel('./test_url.xlsx', index=False)

output

,No,Url
0,1,https://www.1aauto.com/1a-auto-t-shirt-tee-shi...
1,2,https://www.1aauto.com/1a-auto-t-shirt-tee-shi...
2,3,https://www.1aauto.com/1a-auto-t-shirt-tee-shi...
3,4,https://www.1aauto.com/1a-auto-t-shirt-tee-shi...
4,5,https://www.1aauto.com/1a-auto-t-shirt-tee-shi...
5,6,https://www.1aauto.com/1a-auto-t-shirt-tee-shi...


In [7]:
crawler_status = 'ok'

crawler_status

'ok'

In [8]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [9]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_url.xlsx', index=False)

output_error

""
